In [1]:
import shutup
shutup.please()

In [5]:
import gfootball.env as football_env
import numpy as np

import gym
import copy
import tensorflow as tf

from keras.callbacks import TensorBoard
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras import backend as K
from keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()

clipping_val = 0.2
critic_discount = 0.5
entropy_beta = 0.001
gamma = 0.99
lmbda = 0.95


env = gym.make('CartPole-v1')
    
state = env.reset()
state_dims = env.observation_space.shape

state_size = state_dims
n_actions = env.action_space.n
action_space = n_actions
input_shape = env.observation_space.shape

dummy_n = np.zeros((1, 1, n_actions))
dummy_1 = np.zeros((1, 1, 1))

tensor_board = TensorBoard(log_dir='./logs')


target_reached = False
best_reward = 0
iters = 0
max_iters = 50
training_batch=1000
episode = 0
replay_count = 0
shuffle = False
epochs = 10



def get_advantages(values, masks, rewards):
    returns = []
    gae = 0
    for i in reversed(range(len(rewards))):
        delta = rewards[i] + gamma * values[i + 1] * masks[i] - values[i]
        gae = delta + gamma * lmbda * masks[i] * gae
        returns.insert(0, gae + values[i])

    adv = np.array(returns) - values[:-1]
    return returns, (adv - np.mean(adv)) / (np.std(adv) + 1e-10)

def critic_PPO2_loss(values):
    def loss(y_true, y_pred):
        LOSS_CLIPPING = clipping_val
        clipped_value_loss = values + K.clip(y_pred - values, -LOSS_CLIPPING, LOSS_CLIPPING)
        v_loss1 = (y_true - clipped_value_loss) ** 2
        v_loss2 = (y_true - y_pred) ** 2
            
        value_loss = 0.5 * K.mean(K.maximum(v_loss1, v_loss2))
        #value_loss = K.mean((y_true - y_pred) ** 2) # standard PPO loss
        return value_loss
    return loss

def critic_ppo_loss(y_true, y_pred):
    value_loss = K.mean((y_true - y_pred) ** 2) # standard PPO loss
    return value_loss

def actor_ppo_loss(y_true, y_pred):
    # Defined in https://arxiv.org/abs/1707.06347
    advantages, prediction_picks, actions = y_true[:, :1], y_true[:, 1:1+action_space], y_true[:, 1+action_space:]
    LOSS_CLIPPING = clipping_val
    ENTROPY_LOSS = entropy_beta
        
    prob = actions * y_pred
    old_prob = actions * prediction_picks

    prob = K.clip(prob, 1e-10, 1.0)
    old_prob = K.clip(old_prob, 1e-10, 1.0)

    ratio = K.exp(K.log(prob) - K.log(old_prob))
        
    p1 = ratio * advantages
    p2 = K.clip(ratio, min_value=1 - LOSS_CLIPPING, max_value=1 + LOSS_CLIPPING) * advantages

    actor_loss = -K.mean(K.minimum(p1, p2))

    entropy = -(y_pred * K.log(y_pred + 1e-10))
    entropy = ENTROPY_LOSS * K.mean(entropy)
        
    total_loss = actor_loss - entropy

    return total_loss

def get_common_layer(X_input, model="MLP"):
    # Shared CNN layers:
    if model=="CNN":
        X = Conv1D(filters=64, kernel_size=6, padding="same", activation="tanh")(X_input)
        X = MaxPooling1D(pool_size=2)(X)
        X = Conv1D(filters=32, kernel_size=3, padding="same", activation="tanh")(X)
        X = MaxPooling1D(pool_size=2)(X)
        X = Flatten()(X)

    # Shared LSTM layers:
    elif model=="LSTM":
        X = LSTM(512, return_sequences=True)(X_input)
        X = LSTM(256)(X)

    # Shared Dense layers:
    else:
        X = Flatten()(X_input)
        X = Dense(512, activation="relu")(X)
        
    return X

def get_model_actor_simple(input_shape=input_shape, n_actions=n_actions):

    X_input = Input(input_shape)
    X = get_common_layer(X_input)
    X = Dense(512, activation="relu", kernel_initializer=tf.random_normal_initializer(stddev=0.01))(X_input)
    X = Dense(256, activation="relu", kernel_initializer=tf.random_normal_initializer(stddev=0.01))(X)
    X = Dense(64, activation="relu", kernel_initializer=tf.random_normal_initializer(stddev=0.01))(X)
    output = Dense(n_actions, activation="softmax")(X)

    model = Model(inputs = X_input, outputs = output)
    model.compile(loss=actor_ppo_loss, optimizer=Adam(lr=0.00025))
    return model


def get_model_critic_simple(input_shape=input_shape):
    X_input = Input(input_shape)
    V = get_common_layer(X_input)
    V = Dense(512, activation="relu", kernel_initializer='he_uniform')(X_input)
    V = Dense(256, activation="relu", kernel_initializer='he_uniform')(V)
    V = Dense(64, activation="relu", kernel_initializer='he_uniform')(V)
    value = Dense(1, activation=None)(V)

    model = Model(inputs=X_input, outputs = value)
    model.compile(loss=critic_ppo_loss, optimizer=Adam(lr=0.00025))
    return model
    
actor = get_model_actor_simple()
critic = get_model_critic_simple()

def act(state):
    """ example:
    pred = np.array([0.05, 0.85, 0.1])
    action_size = 3
    np.random.choice(a, p=pred)
    result>>> 1, because it have the highest probability to be taken
    """
    # Use the network to predict the next action to take, using the model
    prediction = actor.predict(state)[0]
    action = np.random.choice(n_actions, p=prediction)
    action_onehot = np.zeros([n_actions])
    action_onehot[action] = 1
    return action, action_onehot, prediction

def discount_rewards(reward):#gaes is better
    # Compute the gamma-discounted rewards over an episode
    # We apply the discount and normalize it to avoid big variability of rewards
    gamma = 0.99    # discount rate
    running_add = 0
    discounted_r = np.zeros_like(reward)
    for i in reversed(range(0,len(reward))):
        running_add = running_add * gamma + reward[i]
        discounted_r[i] = running_add

    discounted_r -= np.mean(discounted_r) # normalizing the result
    discounted_r /= (np.std(discounted_r) + 1e-8) # divide by standard deviation
    return discounted_r

def get_gaes(rewards, dones, values, next_values, gamma = 0.99, lamda = 0.9, normalize=True):
    deltas = [r + gamma * (1 - d) * nv - v for r, d, nv, v in zip(rewards, dones, next_values, values)]
    deltas = np.stack(deltas)
    gaes = copy.deepcopy(deltas)
    for t in reversed(range(len(deltas) - 1)):
        gaes[t] = gaes[t] + (1 - dones[t]) * gamma * lamda * gaes[t + 1]

    target = gaes + values
    if normalize:
        gaes = (gaes - gaes.mean()) / (gaes.std() + 1e-8)
    return np.vstack(gaes), np.vstack(target)

def test_reward():
    state = env.reset()
    done = False
    total_reward = 0
   
    limit = 0
    while not done:
        state_input = K.expand_dims(state, 0)
        action_probs = model_actor.predict([state_input, dummy_n, dummy_1, dummy_1, dummy_1], steps=1)
        action = np.argmax(action_probs)
        next_state, reward, done, _ = env.step(action)
        state = next_state
        total_reward += reward
        #print('test reward',reward)
        limit += 1
        if limit > 20:
            break
    print('testing...', total_reward)
    return total_reward

def critic_predict(state):
    return critic.predict([state, np.zeros((state.shape[0], 1))])

def replay(states, actions, rewards, predictions, dones, next_states):
    # reshape memory to appropriate shape for training
    states = np.vstack(states)
    next_states = np.vstack(next_states)
    actions = np.vstack(actions)
    predictions = np.vstack(predictions)

    # Get Critic network predictions 
    values = critic_predict(states)
    next_values = critic_predict(next_states)

    # Compute discounted rewards and advantages
    #discounted_r = self.discount_rewards(rewards)
    #advantages = np.vstack(discounted_r - values)
    advantages, target = get_gaes(rewards, dones, np.squeeze(values), np.squeeze(next_values))
    '''
        pylab.plot(advantages,'.')
        pylab.plot(target,'-')
        ax=pylab.gca()
        ax.grid(True)
        pylab.subplots_adjust(left=0.05, right=0.98, top=0.96, bottom=0.06)
        pylab.show()
        '''
    # stack everything to numpy array
    # pack all advantages, predictions and actions to y_true and when they are received
    # in custom PPO loss function we unpack it
    y_true = np.hstack([advantages, predictions, actions])
        
    # training Actor and Critic networks
    print()
 
    a_loss = actor.fit(states, y_true, epochs=epochs, verbose=0, shuffle=shuffle)
    print('actor loss',np.mean(a_loss.history['loss']))
    print()

    c_loss = critic.fit(states, target, epochs=epochs, verbose=0, shuffle=shuffle)
    print('critic loss',np.mean(c_loss.history['loss']))
    print()
#     self.writer.add_scalar('Data/actor_loss_per_replay', np.sum(a_loss.history['loss']), self.replay_count)
#     self.writer.add_scalar('Data/critic_loss_per_replay', np.sum(c_loss.history['loss']), self.replay_count)
#     self.replay_count += 1
    #replay_count += 1

def run_batch(): # train every self.Training_batch episodes
    scores_ = []
    episodes_ = []
    averages_= [] 
    episode = 0
    state = env.reset()
    state = np.reshape(state, [1,state_size[0]])
    done, score, SAVING = False, 0, ''
    while True:
        # Instantiate or reset games memory
        states, next_states, actions, rewards, predictions, dones = [], [], [], [], [], []
        for t in range(training_batch):
           # env.render()
            # Actor picks an action
            action, action_onehot, prediction = act(state)
                # Retrieve new state, reward, and whether the state is terminal
            next_state, reward, done, _ = env.step(action)
            # Memorize (state, action, reward) for training
            states.append(state)
            next_states.append(np.reshape(next_state, [1, state_size[0]]))
            actions.append(action_onehot)
            rewards.append(reward)
            dones.append(done)
            predictions.append(prediction)
            # Update current state
            state = np.reshape(next_state, [1, state_size[0]])
            score += reward
            if done:
                episode += 1
                SAVING = False
                scores_.append(score)
                averages_.append(sum(scores_[-50:]) / len(scores_[-50:]))

                print('score', averages_[-1])
#                     average, SAVING = self.PlotModel(score, self.episode)
#                     print("episode: {}/{}, score: {}, average: {:.2f} {}".format(self.episode, self.EPISODES, score, average, SAVING))
#                     self.writer.add_scalar(f'Workers:{1}/score_per_episode', score, self.episode)
#                     self.writer.add_scalar(f'Workers:{1}/learning_rate', self.lr, self.episode)

                state, done, score, SAVING = env.reset(), False, 0, ''
                state = np.reshape(state, [1, state_size[0]])
   
        replay(states, actions, rewards, predictions, dones, next_states)
        if episode >= 50000:
            break
    env.close()  




In [6]:

run_batch() # train as PPO, train every epesode

score 17.0
score 24.5
score 24.0
score 22.5
score 22.6
score 20.833333333333332
score 24.285714285714285
score 23.75
score 23.333333333333332
score 21.8
score 20.90909090909091
score 21.5
score 20.923076923076923
score 20.928571428571427
score 21.533333333333335
score 22.25
score 21.88235294117647
score 22.166666666666668
score 21.526315789473685
score 21.65
score 22.857142857142858
score 22.59090909090909
score 22.043478260869566
score 21.833333333333332
score 21.48
score 21.615384615384617
score 21.59259259259259
score 21.678571428571427
score 22.03448275862069
score 22.133333333333333
score 22.483870967741936
score 22.6875
score 22.272727272727273
score 23.294117647058822
score 23.057142857142857
score 23.77777777777778
score 24.135135135135137
score 24.07894736842105
score 23.974358974358974
score 24.2
score 23.951219512195124

actor loss -0.014927236577868463

critic loss 17.086272804641723

score 23.857142857142858
score 23.72093023255814
score 24.068181818181817
score 23.8
score

score 385.44

actor loss -0.003915036481618881

critic loss 6.1429154424250125

score 386.7
score 388.64
score 390.06

actor loss -0.002878206396102905

critic loss 1.7447433293938637

score 392.08
score 394.0

actor loss -0.00046495597697794437

critic loss 5.629956944864988

score 396.06
score 396.52
score 398.26

actor loss -0.005948635548353195

critic loss 0.7154583307862281

score 400.02
score 401.68

actor loss -0.001490634572505951

critic loss 2.3092714490413666

score 402.16
score 400.78
score 402.9

actor loss -0.0020432784885168077

critic loss 17.770918667888644

score 407.54
score 411.24

actor loss -0.0014208008646965026

critic loss 46.90815620909929

score 411.24
score 411.24

actor loss -0.003072922670515254

critic loss 6.184721256166696

score 412.04
score 414.08

actor loss -0.0020099172800779343

critic loss 37.74558903490305

score 414.08
score 417.56

actor loss -0.0027433853626251223

critic loss 40.78235096327662

score 417.56
score 417.56

actor loss -0.00220

score 485.36
score 485.36

actor loss 0.0035703887585550554

critic loss 21.926555338668823

score 485.36
score 494.76

actor loss -0.0002565370261669158

critic loss 34.049020670032505

score 491.72
score 491.72

actor loss -0.007074814484920353

critic loss 6.4911399289608

score 491.72
score 491.72

actor loss 0.0012583609580993653

critic loss 62.801431230902665

score 491.72
score 491.72

actor loss 0.0005380506262183188

critic loss 64.06969417939186

score 491.72
score 491.72

actor loss -0.00014129521846771242

critic loss 64.51605273972154

score 491.72
score 491.72

actor loss -0.0013244397725909948

critic loss 55.77719144678116

score 491.72
score 491.72

actor loss 0.0004416119813919068

critic loss 33.78697878370285

score 491.72
score 491.72

actor loss -0.0008620883822441101

critic loss 36.23656787734032

score 491.72
score 491.72

actor loss -0.0010877979636192323

critic loss 47.16356077003479

score 491.72
score 491.72

actor loss -0.00028585709184408186

critic los

critic loss 75.85760388793945

score 500.0
score 500.0

actor loss -0.005013032966852188

critic loss 69.75478031619191

score 500.0
score 500.0

actor loss -0.002940372171998024

critic loss 74.2475481156349

score 500.0
score 500.0

actor loss -0.0009696186497807503

critic loss 67.5599737195015

score 500.0
score 500.0

actor loss -0.0038835812389850614

critic loss 73.690459445858

score 500.0
score 500.0

actor loss -0.003524840402603149

critic loss 57.775967483472826

score 500.0
score 500.0

actor loss -0.0027570043563842775

critic loss 73.28999654741287

score 500.0
score 500.0

actor loss -0.0028110583990812303

critic loss 65.59129787516595

score 500.0
score 500.0

actor loss -0.0037583687048405416

critic loss 45.43659857237339

score 500.0
score 500.0

actor loss -0.006491848790645599

critic loss 58.83534659686088

score 500.0
score 500.0

actor loss -0.002933766001462936

critic loss 80.4172051407814

score 500.0
score 500.0

actor loss -0.006482432705163956

critic lo

actor loss -0.006933334422111512

critic loss 144.1131493692398

score 466.78
score 466.78

actor loss -0.004861556631140411

critic loss 74.38943484220506

score 466.78
score 467.5

actor loss -0.004003685492277145

critic loss 65.70447715892792

score 467.5
score 467.5

actor loss -0.0068747830006293965

critic loss 69.26757968120576

score 467.5
score 467.5

actor loss -0.004743689296022058

critic loss 64.30339180183412

score 467.5
score 467.5

actor loss -0.002684147393703461

critic loss 57.555275292015075

score 467.5
score 467.5

actor loss -0.004842211243323982

critic loss 65.53772636318207

score 467.5
score 467.5

actor loss -0.0029664146959781645

critic loss 54.343695288944254

score 467.5
score 467.5

actor loss -0.003265733359009028

critic loss 66.7254352606535

score 467.5
score 467.5

actor loss -0.00308437340259552

critic loss 67.41101875023841

score 467.5
score 467.5

actor loss -0.003447883222997188

critic loss 66.05128348238468

score 467.5
score 467.5

actor

In [ ]:
state_size

In [ ]:
type(critic)